# Set 2: *Block crypto*

### Before we get started: The cipher core

The Cryptopals challenges are very keen for AES solutions to be implemented more or less *from scratch*. However, no one is really interested in implementing AES encrpytion in all its detail; it doesn't add that much. On the other hand. implementing different modes in detail *is* important, because that is how you can start to understand the difference between ECB, CBC, and the other modes, and their respective vulnerabilities. Therefore, before we begin, we can write a quick function to act as a AES-cipher core: it should take in a 16-byte buffer and an appropriate-length key, and return the encrypted buffer. Then, ECB, CBC etc. can be constructe from this. For more information on how the AES cipher core works in practice, see the [NIST standard](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.197.pdf) or the [OpenSSL GitHub](https://github.com/openssl/openssl).

In [1]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

def AES_core_encrypt(buffer: bytes, key: bytes) -> bytes:
    assert len(buffer) == 16, ('Cipher core can only handle 16-byte blocks.')
    assert len(key) == 16, ('Key length must be 16 bytes.')
    
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    encryptor = cipher.encryptor()
    return encryptor.update(buffer)

def AES_core_decrypt(buffer: bytes, key: bytes) -> bytes:
    assert len(buffer) == 16, ('Cipher core can only handle 16-byte blocks.')
    assert len(key) == 16, ('Key length must be 16 bytes.')
    
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    decryptor = cipher.decryptor()
    return decryptor.update(buffer)

Let's add a quick sanity check here to make sure what is going on makes good sense, and that decrpytion is, at the least, the opposite action to encryption:

In [2]:
test = b'[SIXTEEN BYTES1]'
test_key = b'YELLOW SUBMARINE'
AES_core_decrypt(AES_core_encrypt(test, test_key), test_key) == test

True

Finally, we can add some useful functions here that we will call A LOT:

In [3]:
def eq_buffer_XOR(a: bytes, b: bytes) -> bytes:
    """
    Take 2 equal length buffers and return their logical XOR.
    """
    assert len(a) == len(b), "Byte buffers must be of equal length."
    return bytes([i^j for i, j in zip(a, b)])

## Task 9: *Implement PKCS#7 padding*

From the assertions in the blocks above, we need to always work with blocks of 16 bytes. The **PKCS#7** scheme achieves this by padding the end of an irregularly sized block with byte representations of the number of bytes needed to bring it up to the correct length. So, if our message to encrypt is 12 bytes, and we want to bring it up to the standard 16, we append 4 bytes of '\x04':

In [4]:
def PKCS7_pad(buffer: bytes, length: int=16) -> bytes:
    if len(buffer) == length:
        return buffer
    
    if not buffer:
        raise ValueError("Can't pad an empty block.")
        
    if len(buffer) > length:
        raise ValueError("Buffer longer than block length.")
    
    to_pad = length - len(buffer)
    return buffer + bytes([to_pad]) * to_pad

def PKCS7_unpad(block: bytes) -> bytes:
    if block[-1] > 16:
        return block
    return block[:-block[-1]]

We can quickly check that we get the expected result:

In [5]:
PKCS7_pad(b'YELLOW SUBMARINE', 20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

In [6]:
PKCS7_unpad(b'YELLOW SUBMARINE\x04\x04\x04\x04')

b'YELLOW SUBMARINE'

## Task 10: *Implement CBC mode*

To implement CBC, we take each successive plaintext block of a message, and, before encrypting it, XOR it with the previous encrpyted block. For the first block, we do the XOR against a (usually) random *initialisation vector* (IV).

In [7]:
def CBC_encrypt(buffer: bytes, key: bytes, IV: bytes, block_length: int=16):
    assert len(IV) == block_length, ("Initialisation vector must be of length equal to 1 block.")
    if len(buffer) <= block_length:
        import warnings
        warnings.warn("Encrypting a block of length <= block_length is NOT secure!")
        return AES_core_encrypt(eq_buffer_XOR(IV, PKCS7_pad(buffer)), key, block_length)
    
    prev_block = IV
    no_blocks = len(buffer) // block_length + 1
    blocks = [buffer[i*16:(i+1)*16] for i in range(no_blocks - 1)]
    try:
        blocks.append(PKCS7_pad(buffer[16*(no_blocks - 1):]))
    except:
        pass
    
    enc_blocks = ["0"*block_length] * no_blocks
    for i, block in enumerate(blocks):
        enc_blocks[i] = AES_core_encrypt(eq_buffer_XOR(prev_block, block), key)
        prev_block = enc_blocks[i]
        
    return b"".join(enc_blocks)

def CBC_decrypt(buffer: bytes, key: bytes, IV: bytes, block_length: int=16):
    assert len(buffer) > block_length, ("Buffer to decrypt is less than a single block.")
    if len(buffer) == block_length:
        return AES_core_decrypt(block, key)
    
    no_blocks = len(buffer) // block_length
    blocks = [buffer[i*16:(i+1)*16] for i in range(no_blocks)]
    
    dec_blocks = enc_blocks = ["0"*block_length] * (no_blocks-1)
    
    for i, block in enumerate(reversed(blocks)):
        if i == 0:
            prev_block = AES_core_decrypt(block, key)
        else:
            dec_blocks[i-1] = eq_buffer_XOR(prev_block, block)
            prev_block = AES_core_decrypt(block, key)
            
    return eq_buffer_XOR(prev_block, IV) + b"".join(reversed(dec_blocks))

In [8]:
ciphertext = ""
with open("s2t2.txt" , 'r') as file:
    for line in file:
        for char in line:
            if char == "\n":
                continue
            else:
                ciphertext += char
                
import base64
ciphertext_bytes = base64.b64decode(ciphertext)

In [9]:
IV = b'\x00' * 16
decryption = CBC_decrypt(ciphertext_bytes, b'YELLOW SUBMARINE', IV)

In [10]:
print(decryption.decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


N.B. these characters at the end are just the PKCS#7 padding - we don't need to worry about them.

## Task 11: *An ECB/CBC detection oracle*

The first stop here is actually to write some code to do arbitrary ECB encryption and decrpytion. *Yes*, this is technically the job of Set 1 Task 7, but hey, we're learning on the job. First, a quick function so that from now on, we can more easily break up a message into the appropriate blocks:

In [11]:
from typing import List
from itertools import pairwise

def blocks(buffer: bytes, blocksize: int=16) -> List[bytes]:
    no_of_blocks = len(buffer) // blocksize
    if len(buffer) % blocksize == 0:
        return [buffer[blocksize*i:blocksize*(i+1)] for i in range(no_of_blocks)]
    blocks = [buffer[blocksize*i:blocksize*(i+1)] for i in range(no_of_blocks)]
    blocks.append(PKCS7_pad(buffer[16*no_of_blocks:]))
    return blocks

def sanitise_decryption(blocks: List[bytes]) -> str:
    return b"".join(blocks[:-1] + [PKCS7_unpad(blocks[-1])])

def ECB_encrypt(buffer: bytes, key: bytes, blocklength: int=16) -> bytes:
    return b"".join([AES_core_encrypt(block, key) for block in blocks(buffer)])

def ECB_decrypt(buffer: bytes, key: bytes, blocklength: int=16) -> bytes:
    return sanitise_decryption([AES_core_decrypt(block, key) for block in blocks(buffer)])

def CBC_encrypt(buffer: bytes, key: bytes, IV: bytes, block_length: int=16):
    prev_block = IV
    bks = blocks(buffer)
    
    enc_blocks = [(prev_block := AES_core_encrypt(eq_buffer_XOR(prev_block, block), key)) for block in bks]
    return b"".join(enc_blocks)

def CBC_decrypt(buffer: bytes, key: bytes, IV: bytes, block_length: int=16):
    bks = blocks(IV + buffer)
    
    dec_blocks = [eq_buffer_XOR(AES_core_decrypt(prev_block, key), block) for prev_block, block in pairwise(reversed(bks))]
    return b"".join(reversed(dec_blocks))

In [12]:
ciphertext = ""
with open('s1t7_data.txt', 'r') as file:
    for line in file:
        for char in line:
            if char == "\n":
                continue
            ciphertext += char
cipher_bytes = base64.b64decode(ciphertext)

In [13]:
print(ECB_decrypt(cipher_bytes, b'YELLOW SUBMARINE').decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


I'll leave that here as a quick test that the ECB code is definitelty working OK!

Now, we want to write an encryption oracle that:
- encodes using a random, unknown key.
- uses ECB or CBC half the time each, chosen randomly.
- pad 5-10 (count chosen randomly) bytes before and after the plaintext.

In [14]:
from secrets import token_bytes, randbelow

def encryption_oracle(plaintext: str) -> str:
    ptext_bytes = plaintext.encode()
    rand_padding1 = b'\x00' * (5 + randbelow(6))
    rand_padding2 = b'\x00' * (5 + randbelow(6))
    padded_ptext = rand_padding1 + ptext_bytes + rand_padding2
    random_key = token_bytes(16)
    
    if randbelow(2) == 0:
        """
        Use ECB mode:
        """
        return base64.b64encode(ECB_encrypt(padded_ptext, random_key)).decode()
    
    """
    Use CBC mode:
    """
    random_IV = token_bytes(16)
    return base64.b64encode(CBC_encrypt(padded_ptext, random_key, random_IV)).decode()

In [15]:
def repeats_in(ciphertext: str, blocklength: int=16) -> bool:
    cipher_bytes = base64.b64decode(ciphertext)
    bks = blocks(cipher_bytes)
    if len(set(bks)) == len(bks):
        return False
    return True

In [16]:
malicious_plaintext= "A" * 100
repeats_in(encryption_oracle(malicious_plaintext))

True

By using a malicious plaintext, even with the padding at the start and end we can find repeats and thus verify if we're in ECB or CBC mode! Now let's wrap this up so that we can write a function that takes the encryption oracle as an input, and tells us if its ECB or CBC:

In [17]:
class Oracle():
    my_modes={0:'ECB', 1:'CBC'}
    def __init__(self, blocklength: int=16):
        self.blocklength = 16
        self.mode = self.my_modes[randbelow(2)]
        
    def __repr__(self):
        return f"Encrpytion oracle that uses {self.mode}."
    
    def encrypt(self, plaintext: str) -> str:
        random_key = token_bytes(16)
        pad1 = token_bytes(randbelow(6)+5)
        pad2 = token_bytes(randbelow(6)+5)
        ptext_pb = pad1 + plaintext.encode() + pad2
        match self.mode:
            case 'ECB':
                return base64.b64encode(ECB_encrypt(ptext_pb, random_key)).decode()
            case 'CBC':
                random_IV = token_bytes(16)
                return base64.b64encode(CBC_encrypt(ptext_pb, random_key, random_IV)).decode()
            case _:
                raise ValueError(f"Unrecognised encryption type '{self.mode}'.")
        

In [18]:
a = Oracle()
print(a.mode, enc := a.encrypt(malicious_plaintext), repeats_in(enc))

def detection_oracle(encryptor: callable) -> str:
    malicious_plaintext = "A" * 100
    if repeats_in(encryptor(malicious_plaintext)):
        return 'ECB'
    return 'CBC'

print(detection_oracle(a.encrypt))

ECB Cqx8vXfjBnzBGBNRoJtmCsd4ecFBx2fzp3YK24vEy6LHeHnBQcdn86d2CtuLxMuix3h5wUHHZ/Ondgrbi8TLosd4ecFBx2fzp3YK24vEy6LHeHnBQcdn86d2CtuLxMuiRuVyfpt/wDZlUcWDeXxA/XXq4s1bqjEVKubwlE5qBZk= True
ECB


## Task 12: *Byte-at-a-time ECB decryption (Simple)*

first, we need a function that encrypts buffers under a consistent but unknown key:

In [19]:
consistent_key = token_bytes(16)  # Don't re-run this cell while the analysis is going!

In [20]:
def consistent_ECB(buffer: bytes) -> bytes:
    return ECB_encrypt(buffer, consistent_key)

In [21]:
unknown_string = """Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK""".replace("\n", "")

In [22]:
def attack_ECB(prefix: bytes) -> bytes:
    return consistent_ECB(prefix + b"".join(blocks(base64.b64decode(unknown_string))))

In [23]:
attack_ECB(b"")

b'\xcf\xc2\x1cZpH\x8a\x8c%\xebB\x83\xa2\xe3\xe5yD \x0c,"\xc6O\xb8\xa9\x12\x10\x8a\xd5\x15\xce\x99\x9e\xd5p\xe9\xe7\xd0,\xa4\x17\xed\x0e\x8e*O5\x0eLi\xed\xab\xa8\x9a\xba\xbd\x8f\xa2\xeb\x1a\xccu\xc2\x7fy\xc2\xdc-\xd1\xbe\xa5?\xe8\xcazp*9\xe96\xbe\x8d\n\x1f\xb3Xb\xf7\xc6+v\xffC\x06:~?\x0f\xf6\x02\x82so\x8a\xab(.\xe5~\xe7\x00\x1c\xab/\xf6\xbe\xac~;w\xc1\xf5\xce~\xce|\xd7*&\xcc\xf3<\x03,\x8e\x88\xfe\x87\xc7\x1fR\xab\xe9\xff'

The above gibberish represents the raw, encrypted text - now we want to see what we can discover by prepending malicious plaintext.

In [24]:
raw_encrypted = attack_ECB(b'')

In [25]:
for i in range(1, 100):
    if raw_encrypted in attack_ECB(b'0' * i):
        print(f"The block length is {i}")
        break

The block length is 16


This effectively also verifies that the encryption is in fact in ECB mode; because successive blocks don't affect each other; prepending a whole block leaves the rest of the text unchanged.

Now lets start to work out what the plaintext actually is. If we prepend 7 b'0's, then the 8th byte of the block will be the first byte of the plaintext:

In [26]:
malicious_block_1 = attack_ECB(b'0' * 15)[:16]
print(malicious_block_1)

b'=!t\xe7\xe0Q\xcdX\xf1\xa2\x85\x82\x1e@D\x91'


In [27]:
decryption_dictionary = {attack_ECB(b'0' * 15 + bytes([a]))[:16] : bytes([a]) for a in range(256)}
decryption_dictionary[malicious_block_1]

b'R'

And so we find the first byte of the plaintext! For the next one, the task is very similar: because we now know the first character, we can include it in our malicious plaintext, so that we're now investigating the second:

In [28]:
malicious_block_2 = attack_ECB(b'0' * 14)[:16]
print(malicious_block_2)

b'k\xaeX\xefN\xa2:\xc1P\xb4\xf6$\xa3*\xd1\xc9'


In [29]:
decryption_dictionary = {attack_ECB(b'0' * 14 + b'R' + bytes([a]))[:16] : bytes([a]) for a in range(256)}
decryption_dictionary[malicious_block_2]

b'o'

Let's extend and automate this for the entire first byte:

In [30]:
known_bytes = b''
for i in range(16):
    malicious_block = attack_ECB(b'0' * (15-i))[:16]
    decryption_dictionary = {attack_ECB(b'0' * (15-i) + known_bytes + bytes([a]))[:16] : bytes([a]) for a in range(256)}
    known_bytes += decryption_dictionary[malicious_block]
    
print(known_bytes)

b"Rollin' in my 5."


Not bad. Now how do we extend this for the next block? We want to end up in a situation where we can test the bytes of the next block one at a time. So, we alter our padding so that i.e. the first byte of the second block falls as the last block of an encrpyted block, thus allowing us to guess its value using the dictionary comparison method above.

In [31]:
attack_ECB(b'')[:16]

b'\xcf\xc2\x1cZpH\x8a\x8c%\xebB\x83\xa2\xe3\xe5y'

In [32]:
attack_ECB(b'0' * 15)[16:32]

b'\x02\xdb\x98E\xfd|\xd8\x9f\xba\xcb\xcek\x0e\xfa\x8d\xf0'

In [33]:
decryption_dictionary = {attack_ECB(b'0' * 15 + known_bytes + bytes([a]))[16:32] : bytes([a]) for a in range(256)}
decryption_dictionary[attack_ECB(b'0' * 15)[16:32]]

b'0'

Nice! that works, so now we can move to automate this for the whole message. It's worth bearing in mind that we only ever need the 7 preceding bytes to the new character, so we don't need to feed in the whole of the decrypted message.

In [34]:
divmod(len(attack_ECB(b'')), 16)

(9, 0)

In [35]:
prev_block = known_bytes
blocks(attack_ECB(b''))

[b'\xcf\xc2\x1cZpH\x8a\x8c%\xebB\x83\xa2\xe3\xe5y',
 b'D \x0c,"\xc6O\xb8\xa9\x12\x10\x8a\xd5\x15\xce\x99',
 b'\x9e\xd5p\xe9\xe7\xd0,\xa4\x17\xed\x0e\x8e*O5\x0e',
 b'Li\xed\xab\xa8\x9a\xba\xbd\x8f\xa2\xeb\x1a\xccu\xc2\x7f',
 b'y\xc2\xdc-\xd1\xbe\xa5?\xe8\xcazp*9\xe96',
 b'\xbe\x8d\n\x1f\xb3Xb\xf7\xc6+v\xffC\x06:~',
 b'?\x0f\xf6\x02\x82so\x8a\xab(.\xe5~\xe7\x00\x1c',
 b'\xab/\xf6\xbe\xac~;w\xc1\xf5\xce~\xce|\xd7*',
 b'&\xcc\xf3<\x03,\x8e\x88\xfe\x87\xc7\x1fR\xab\xe9\xff']

In [36]:
known_bytes = b''
for i in range(16):
    malicious_block = attack_ECB(b'0' * (15-i))[16:32]
    decryption_dictionary = {attack_ECB(b'0' * (15-i) + prev_block + known_bytes + bytes([a]))[16:32] : bytes([a]) for a in range(256)}
    known_bytes += decryption_dictionary[malicious_block]
    
print(known_bytes)

b'0\nWith my rag-to'


That looks like real progress. Now, let's wrap this all up in to loop over the blocks of the ciphertext:

In [37]:
ciphertext = attack_ECB(b'')
known_bytes = b"Rollin' in my 5."
prev_block = b"Rollin' in my 5."
for b in range(1, len(ciphertext) // 16):
    print(prev_block)
    for i in range(16):
        malicious_block = attack_ECB(b'0' * (15-i))[16*b:16*(b+1)]
        decryption_dictionary = {attack_ECB(b'0' * (15-i) + prev_block + known_bytes[b*16:] + bytes([a]))[16:32] : bytes([a]) for a in range(256)}
        # print(decryption_dictionary[malicious_block])
        known_bytes += decryption_dictionary[malicious_block]
    prev_block = known_bytes[-16:]
    # print(prev_block)

b"Rollin' in my 5."
b'0\nWith my rag-to'
b'p down so my hai'
b'r can blow\nThe g'
b'irlies on standb'
b'y waving just to'
b' say hi\nDid you '
b'stop? No, I just'


In [38]:
print(known_bytes.decode())

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



Nice. N.B. padding on the last block almost threw you off.

## Task 13: *ECB cut-and-paste*

In [39]:
def kvparse(cookie: str) -> dict:
    pairs = [p.split("=") for p in cookie.split("&")]
    return {k:v for k, v in pairs}

In [40]:
kvparse('foo=bar&baz=qux&zap=zazzle')

{'foo': 'bar', 'baz': 'qux', 'zap': 'zazzle'}

In [41]:
USERS = randbelow(1000)

In [42]:
import warnings
def make_profile(user: str) -> str:
    if '=' in user or '&' in user:
        warnings.warn(Warning("Encoding metacharacters used in user input, removing..."))
        return make_profile(user.replace("&", "").replace("=", ""))
    global USERS
    user_dict = {'email':user, 'ID':str(USERS), 'role':'user'}
    keypairs = [k + '=' + v for k, v in user_dict.items()]
    update_users()
    return "&".join(keypairs)

def update_users():
    global USERS
    # USERS += 1

In [43]:
AES_key = token_bytes(16)

In [44]:
def profile_for(user: str) -> str:
    global AES_key
    return base64.b64encode(ECB_encrypt(make_profile(user).encode(), AES_key)).decode()

In [45]:
profile_for('harry@fox.com')

'tNe5bpEflPnnzVbBhW+rhbpwlKAeUV5DsCDnJOlG5u1TxOpr4w9Gqc6NZE3ANSrR'

In [46]:
def serverside(cookie: str) -> dict:
    global AES_key
    return kvparse(ECB_decrypt(base64.b64decode(cookie), AES_key).decode())

In [47]:
serverside(profile_for('harry@fox.com'))

{'email': 'harry@fox.com', 'ID': '967', 'role': 'user'}

With that background out of the way our goal is as follows: to make an admin profile by working out how its encrypted cookie must look. For now, let's freeze the USERS increment so that we only have 1 moving part to our equation. First, let's take a look at the encrypted cookie for 'eve@evil.net'.

In [48]:
profile_for('eve@evil.net')

'uw8Q2YL9s8DF8BCc7SUwXm9wcuHAraab6g64btnUpFcBKBBzK7J6tQ9Wlu72Cx6e'

Let's break down this cookie a little. The first bit of the first byte is the encrypted 'email' key, followed by an equals sign, and then the user input *that we control*. Subsequently, there is the ID, which contains some unknown number, and then the 'role' field. First of all, let's work out what the final byte of the cookie would look like, encrypted, if it begun exactly on user. Including the padding, this would be plaintext 'user\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c', wich we can artificially place in the second byte of a dummy cookie like so:

In [49]:
user_suffix = base64.b64decode(profile_for('A' * 10 + 'user' + 12 * '\x0c' + '@gmail.com'))[16:32]
user_suffix

b'S\xc4\xeak\xe3\x0fF\xa9\xce\x8ddM\xc05*\xd1'

Check that this is *only* the stuff we want by modifying the 'A' to some other letter and the gmail hande to something else.

Now, we want to make a cookie so that the above suffix is the whole of the last byte. We can achieve this by modifying the length of our user input until we get a collision:

In [53]:
user_input = ''
while len(user_input) < 20:
    cookie = base64.b64decode(profile_for(user_input))
    if user_suffix in cookie:
        print(len(user_input))
    user_input += 'A'

13


So a length 13 input will give us the last byte in the correct place, ready to be swapped out. The last thing to do is to work out what to swap it out with:

In [54]:
admin_suffix = base64.b64decode(profile_for('A' * 10 + 'admin' + 11 * '\x0b' + '@gmail.com'))[16:32]

Then finally we should be able to make a 13-character email which we can transform into an admin-permissioned cookie!

In [60]:
prefix = base64.b64decode(profile_for("eve@gmail.com"))[:-16]
cookie = base64.b64encode(prefix + admin_suffix).decode()
serverside(cookie)

{'email': 'eve@gmail.com', 'ID': '967', 'role': 'admin'}

Nice!

## Task 14: *Byte-at-a-time ECB decryption (Harder)*